In [1]:
import numpy as np
import pandas as pd
import re

In [5]:
master_folder = "/home/charly/Desktop/Wunderman/mazda/MailChimp/"
lists = "lists/"
granular_activity = "granular_activity/"
geo_data = "geo_data/"
campaigns_content = "campaigns_content/"
aggregate_activity = "aggregate_activity/"
campaigns = "campaigns.csv"

In [52]:
df_campaigns = pd.read_csv(master_folder + campaigns)

In [6]:
df_cust_data = pd.read_excel('mazda_dataset.xlsx','Customer data')
df_campaign_customers = pd.read_excel('mazda_dataset.xlsx','Campaigns_customers')
df_prospects_data = pd.read_excel('mazda_dataset.xlsx','Prospects data')
df_campaign_prospects = pd.read_excel('mazda_dataset.xlsx','Campaigns_prospects')

In [7]:
# Functions to count the number of files per folder
import os, os.path
def filecount(dir_name):
    return len([f for f in os.listdir(dir_name) if os.path.isfile(dir_name +f)])

## Granular Activity

Number of rows in each of the files in the granular activity folder corresponds to either "Total Clicks" or "Total Opens" in the campaign master file.

Clicks files contain three columns:
- Timestamp
- Email
- Url

Opens files contain two columns:
- Timestamp
- Url

In [8]:
#filecount
print("# of files in clicks :", filecount(os.fspath(master_folder + granular_activity + "clicks/")))
print("# of files in opens :", filecount(os.fspath(master_folder + granular_activity + "opens/")))

# of files in clicks : 183
# of files in opens : 183


In [9]:
files_granu = [f for f in os.listdir(master_folder + granular_activity +  "clicks/")]

In [10]:
df_gran_act_clicks_rand = pd.read_csv(master_folder + granular_activity + \
                  "clicks/" + files_granu[0], low_memory=False)
df_gran_act_opens_rand = pd.read_csv(master_folder + granular_activity + \
                  "opens/" + files_granu[0], low_memory=False)

In [11]:
df_gran_act_clicks_rand.head()

,Timestamp,Email,Url
0,2018-01-08 14:24:12,Pascal.Kaufhold@Post.Lu,http://www.mazda.lu
1,2018-01-08 14:24:34,Serge.Milbert@Bil.Com,http://mazda.lu
2,2018-01-08 14:25:32,Stathisko@Hotmail.Com,http://www.mazda.lu
3,2018-01-08 14:28:06,Elise.Gagliano@Gmail.Com,http://www.mazda.lu
4,2018-01-08 14:28:32,Ricardo.Pimentel85@Gmail.Com,http://www.mazda.lu


In [12]:
df_gran_act_opens_rand.head()

,Timestamp,Email
0,2018-01-08 14:23:15,Pgomes@Proactif.Lu
1,2018-01-08 14:23:37,Chris@Mind.Lu
2,2018-01-08 14:23:38,Pacerioni@Gmail.Com
3,2018-01-08 14:23:38,Ybagsnaiw@Hotmail.De
4,2018-01-08 14:23:43,Christine.Thill@Gmail.Com


In [13]:
# Join all the granular_activity together
def join_granular():
    df = {}
    df_join = pd.DataFrame()
    for i in ['clicks/', 'opens/']:
        df[i] = pd.DataFrame() # df[i]
        counter = 0
        for j in [f for f in os.listdir(master_folder + granular_activity +  i)]:
            temp_df = pd.read_csv(master_folder + granular_activity +  i + j)
            temp_df['file_name'] = j
            temp_df['type_list'] = i[:-1]
            if counter == 0:
                df[i] = temp_df
                counter += 1
            else:
                df[i] = pd.concat([df[i], temp_df])
                counter += 1
        if counter == 1:
            df_join = df[i]
        else:
            df_join = pd.concat([df_join, df[i]])
            
    df_join.reset_index(inplace = True)
    df_join.drop(['index'],axis=1,inplace = True)
    return df_join

In [14]:
df_granular = join_granular()

In [16]:
# Make sure the email addresses are well formatted
df_granular['Email Address'] = df_granular['Email'].apply(lambda x: str(x.lower().strip()))
df_granular['Email Address'] = df_granular['Email Address'].astype(str)
df_granular.drop('Email', axis = 1, inplace = True)

## Geo Data

Only shows data for users where geo data was enabled. For this reason, number of clicks and opens might not correspond to total number of clicks and opens on the master campaign file.

Not of interest as aggregate data per country and not per user

In [18]:
#filecount
print("# of files in clicks :", filecount(os.fspath(master_folder + geo_data + "clicks/")))
print("# of files in opens :", filecount(os.fspath(master_folder + geo_data + "opens/")))

# of files in clicks : 183
# of files in opens : 183


In [19]:
files_geo = [f for f in os.listdir(master_folder + geo_data +  "clicks/")]

In [20]:
df_geo_clicks_rand = pd.read_csv(master_folder + geo_data + \
                  "clicks/" + files_geo[0], low_memory=False)

In [21]:
df_geo_clicks_rand.columns

Index(['Country Code', 'Region', 'Url', 'Clicks'], dtype='object')

In [22]:
df_geo_clicks_rand.head()

,Country Code,Region,Url,Clicks
0,BE,BE_BRU,http://mazda.lu,1
1,BE,BE_WHT,http://mazda.lu,1
2,DE,BE_BY,http://www.mazda.lu,1
3,FR,FR_J,http://www.mazda.lu,1
4,HU,NaN,http://www.mazda.lu,2


In [24]:
df_geo_opens_rand = pd.read_csv(master_folder + geo_data + \
                  "opens/" + files_geo[0], low_memory=False)

In [25]:
df_geo_opens_rand.columns

Index(['Country Code', 'Region', 'Opens'], dtype='object')

In [26]:
df_geo_opens_rand.head()

,Country Code,Region,Opens
0,BE,BE_BRU,22
1,BE,BE_VOV,4
2,BE,BE_WBR,2
3,BE,BE_WHT,6
4,BE,BE_WLX,1


## campaigns_content

HTML and text files containing the contents of each campaign

In [27]:
list_files = [f for f in os.listdir(os.fspath(master_folder + campaigns_content)) \
              if os.path.isfile(os.fspath(master_folder + campaigns_content) +f)]

In [28]:
html_arr = []
txt_arr = []
other_arr = []
for i in list_files:
    if i.split(".")[-1] == 'html':
        html_arr.append(i)
    elif i.split(".")[-1] == 'txt':
        txt_arr.append(i)
    else:
        other_arr.append(i)

In [29]:
#filecount
print("# of files in campaigns_content :", filecount(os.fspath(master_folder + campaigns_content)))
print("# of html files :", len(html_arr))
print("# of txt files :", len(txt_arr))
print("# of other files :", len(other_arr))

# of files in campaigns_content : 376
# of html files : 188
# of txt files : 188
# of other files : 0


In [35]:
html_arr[0]

'779765_lancering-cx-5-barcelona-nl-extra-intern.html'

In [40]:
#Script to extract only text from the content html files

import urllib.request
from bs4 import BeautifulSoup
import re

with urllib.request.urlopen('file:///home/charly/Desktop/Wunderman/mazda/MailChimp/campaigns_content/' + \
                           html_arr[25]) as url:
    s = url.read()
    soup = BeautifulSoup(s, 'html.parser')
    texts = soup.findAll(text=True)

def visible(element):
    if element.parent.name in ['style', 'script', 'document', 'head', 'title']: return False
    elif re.match('<!--.*-->', str(element)): return False
    elif re.match('\n', str(element)): return False
    elif re.match('^HTML', str(element)): return False
    elif re.match('^[\[<]', str(element)): return False
    elif re.search('\xa0',  str(element)): return False
    elif re.search('<',  str(element)): return False
    elif re.search('\|',  str(element)): return False
    elif re.search('© | SNIPPETS | CTA | This message was sent | Please do not reply | PREVIEW | START | END | Android Google App Fix Start | Android Google App Fix End',\
                   str(element)): return False
    elif len(element) < 4: return False
    else: 
        return True
    
def replace(element):
    if re.search(u'\r\n', str(element)): 
        return element.replace(u'\r\n', u' ')

visible_texts = list(filter(visible, texts))
map_texts = list(map(lambda x: x.replace(u'\r\n', u' ').strip(), visible_texts))

sub = 'Deze e-mail werd verstuurd'

for i, k in enumerate(map_texts):
    if sub in k:
        index = i

# map_texts = map_texts[:index]

print(map_texts)

['start mail', 'Lees online', 'Sinds woensdag 1 februari is het zover! Antwerpen weert de vervuilendste wagens uit het centrum. Want de lokale luchtkwaliteit moet erop vooruit. Voor het milieu én voor de gezondheid van de inwoners en bezoekers.', 'SWITCH CHEQUE T.W.V. € 500 *', 'Haalt jouw auto de norm niet en wil je met een schoon geweten de stad in?', 'Ruil je oude voertuig in voor een nieuwe Mazda en geniet met de Switch Cheque € 500 extra korting op de aankoopprijs.', 'Kom in februari vrijblijvend langs bij een van onze zes deelnemende verdelers onderaan deze mail en vraag om de Switch Cheque*.', 'MAZDA EMISSIE CHECKPUNT', 'Tijdens de eerste dag van de lage-emissiezone installeerde Mazda een Emissie Checkpunt op de parking van deSingel. Daar kon je controleren of je auto Antwerpen nog binnen mag. Voldoet hij niet aan de norm? Dan kon je hem voor een dag omruilen tegen een gloednieuwe Mazda.', 'MAG JOUW AUTO NOG BINNEN?', 'Op de website van de stad Antwerpen krijg je meteen antwoord

In [43]:
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('dutch'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in map_texts] 

In [44]:
# Importing Gensim
import gensim
from gensim import corpora
from gensim.corpora import Dictionary

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.029*"mazda" + 0.021*"03" + 0.021*"auto"'), (1, '0.029*"cheque" + 0.018*"geldig" + 0.018*"automotive"'), (2, '0.020*"03" + 0.011*"antwerpen" + 0.011*"februari"')]


In [45]:
from nltk.corpus import words

correct_spellings = words.words()

In [46]:
correct_spellings

['A',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'Aani',
 'aardvark',
 'aardwolf',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'aba',
 'Ababdeh',
 'Ababua',
 'abac',
 'abaca',
 'abacate',
 'abacay',
 'abacinate',
 'abacination',
 'abaciscus',
 'abacist',
 'aback',
 'abactinal',
 'abactinally',
 'abaction',
 'abactor',
 'abaculus',
 'abacus',
 'Abadite',
 'abaff',
 'abaft',
 'abaisance',
 'abaiser',
 'abaissed',
 'abalienate',
 'abalienation',
 'abalone',
 'Abama',
 'abampere',
 'abandon',
 'abandonable',
 'abandoned',
 'abandonedly',
 'abandonee',
 'abandoner',
 'abandonment',
 'Abanic',
 'Abantes',
 'abaptiston',
 'Abarambo',
 'Abaris',
 'abarthrosis',
 'abarticular',
 'abarticulation',
 'abas',
 'abase',
 'abased',
 'abasedly',
 'abasedness',
 'abasement',
 'abaser',
 'Abasgi',
 'abash',
 'abashed',
 'abashedly',
 'abashedness',
 'abashless',
 'abashlessly',
 'abashment',
 'abasia',
 'abasic',
 'abask',
 'Abassin',
 'abastardize',
 'abatable',
 'abate

## Match between master campaign file and csv files

In [48]:
agg_click_list = [f for f in os.listdir(master_folder +'aggregate_activity/click_activity/')\
 if os.path.isfile(master_folder + 'aggregate_activity/click_activity/' +f)]

In [49]:
agg_click_list

['340577_mazda-mazdafee7w463-mazda-cheque-lux.csv',
 '996581_september-newsletter-fr-customer-3j.csv',
 '467741_20160909-newsletter-september-prospecten-fr-combo-1.csv',
 '996485_september-newsletter-nl-prospect.csv',
 '425061_20160818-verliezers-24u-zolder-fr.csv',
 '467873_20160909-newsletter-september-mx-5-fr.csv',
 '792705_lancering-cx-5-barcelona-fr-extra-intern-copy-02-.csv',
 '380077_e-newsletter-juli-klanten-meer-dan-3-jaar-fr.csv',
 '781461_cx-3-klanten-nl-test-cta.csv',
 '196233_2016-may-newsletter-3y-benl-final-.csv',
 '522305_mazda-mania-verliezersmail-nl.csv',
 '784981_lancering-cx-5-barcelona-nl-extra-intern-copy-02-.csv',
 '773573_test-upgrade-form-nils-copy-01-.csv',
 '340609_mazda-mazdhoc7w495-newsletter-januari-nl-3-jaar-plus-wedstrijd.csv',
 '414381_20160812-verliezers-open-air-b-b-nl-combo-2.csv',
 '380685_e-newsletter-juli-klanten-minder-dan-3-jaar-nl-test-brochure-kathy-.csv',
 '996585_september-newsletter-fr-customer-3j.csv',
 '781513_cx-3-klanten-fr-test-cta.csv

In [53]:
import re

#function to find the row in the campaign master file based on name of individual files in other folder

def find_row(file_searched):
    file = re.split("[_\-.]",file_searched)
    regex1 = re.compile("(^[0-9]+$)")
    regex2 = re.compile("(csv)")
    filtered = list(filter(lambda i: not regex1.search(i), file))
    filtered = list(filter(lambda i: not regex2.search(i), filtered))
    regexp = re.compile('|'.join(filtered),re.IGNORECASE)
    
    return df_campaigns[df_campaigns['Title'].apply( lambda x: len(re.findall(regexp, x)) == len(filtered))]\
    .index.values

In [54]:
find_row(agg_click_list[0])

array([10])

In [57]:
df_campaigns.iloc[10].Title

'Mazda - MAZDAFEE7W463 - Mazda cheque LUX'

In [55]:
match_file_campaign = []

for i in range(len(agg_click_list)):
    match_file_campaign.append(find_row(agg_click_list[i]))

for i, k in enumerate(match_file_campaign):
    if len(match_file_campaign[i]) == 0:
        print(agg_click_list[i])

792705_lancering-cx-5-barcelona-fr-extra-intern-copy-02-.csv
196233_2016-may-newsletter-3y-benl-final-.csv
784981_lancering-cx-5-barcelona-nl-extra-intern-copy-02-.csv
773573_test-upgrade-form-nils-copy-01-.csv
380685_e-newsletter-juli-klanten-minder-dan-3-jaar-nl-test-brochure-kathy-.csv
792777_lancering-cx-5-barcelona-fr-extra-intern-copy-03-.csv
781541_lancering-cx-5-barcelona-nl-extra-intern-copy-01-.csv
873301_nieuwsbrief-mei-customer-3y-benl-.csv
789601_lancering-cx-5-barcelona-nl-extra-intern-copy-03-.csv
250741_2016-june-newsletter-prospect-extra-benl-final-.csv
199473_2016-may-newsletter-prospect-benl-final-.csv
825073_cx-5-wedstrijd-barcelona-update-form-nl-copy-01-.csv
633709_mazda-autosalon-fleet-fr-copy-01-.csv
198773_2016-may-newsletter-3y-befr-final-.csv
878765_nieuwsbrief-mei-customer-3y-luxfr-copy-02-.csv
198821_2016-may-newsletter-3y-befr-final-.csv
633713_mazda-autosalon-fleet-nl-copy-01-.csv
792669_lancering-cx-5-barcelona-nl-extra-intern-copy-06-.csv
210173_2016-ma